# Essential basic functionality

In [1]:
import numpy as np
import pandas as pd

In [3]:
#index = pd.date_range('2019/08/15/', periods=8)
#index = pd.date_range('08/15/2019', periods=8)
#index = pd.date_range('15/08/2019', periods=8)
index = pd.date_range('20190815', periods=8)
index

DatetimeIndex(['2019-08-15', '2019-08-16', '2019-08-17', '2019-08-18',
               '2019-08-19', '2019-08-20', '2019-08-21', '2019-08-22'],
              dtype='datetime64[ns]', freq='D')

####  pd.date_range('2019/15/08', periods=8) - ValueError: could not convert string to Timestamp

In [19]:
s = pd.Series(np.random.randn(5), index=['a', 'b', 'c', 'd', 'e'])
df = pd.DataFrame(np.random.randn(8, 3), index=index, columns=['A', 'B', 'C'])

## Head and tail

To view a small sample of a Series or DataFrame object, use the head() and tail() methods. The default number of elements to display is five, but you may pass a custom number.

In [20]:
long_series = pd.Series(np.random.randn(1000))

In [21]:
long_series.head()

0    0.791854
1    0.005587
2   -0.420022
3   -0.195188
4   -0.966391
dtype: float64

In [22]:
long_series.tail()

995    1.162748
996    1.449234
997   -1.721776
998   -0.478430
999   -0.035263
dtype: float64

In [23]:
long_series.head(3)

0    0.791854
1    0.005587
2   -0.420022
dtype: float64

## Attributes and underlying data

pandas objects have a number of attributes enabling you __to access the metadata__
#### shape: gives the axis dimensions of the object, consistent with ndarray
#### Axis labels - 1. Series: index (only axis), 2. DataFrame: index (rows) and columns

In [24]:
df[:2]

,A,B,C
2019-08-15,-1.108985,0.171525,-0.743978
2019-08-16,-0.058313,-0.267974,0.682075


In [25]:
df.columns = [x.lower() for x in df.columns]

In [26]:
df.head()

,a,b,c
2019-08-15,-1.108985,0.171525,-0.743978
2019-08-16,-0.058313,-0.267974,0.682075
2019-08-17,0.897023,0.123224,-0.507456
2019-08-18,-0.767732,-0.427380,-0.040074
2019-08-19,2.089524,-0.523301,-0.605807


Pandas objects (Index, Series, DataFrame) can be thought of as containers for arrays, which hold the actual data and do the actual computation. For many types, the underlying array is a numpy.ndarray. However, pandas and 3rd party libraries may extend NumPy’s type system to add support for custom arrays (see dtypes).

#### To get the actual data inside a Index or Series, use the .array property

In [28]:
s.array

AttributeError: 'Series' object has no attribute 'array'

<PandasArray>
[ 0.4691122999071863, -0.2828633443286633, -1.5090585031735124,-1.1356323710171934,  1.2121120250208506]
Length: 5, dtype: float64

In [29]:
s.index.array

AttributeError: 'Index' object has no attribute 'array'

<PandasArray>
['a', 'b', 'c', 'd', 'e']
Length: 5, dtype: object

#### When the Series or Index is backed by an ExtensionArray, to_numpy() may involve copying data and coercing values. 

array will always be an __ExtensionArray__

If you know you need a NumPy array, use __to_numpy() or numpy.asarray()__.

In [32]:
s.to_numpy()

AttributeError: 'Series' object has no attribute 'to_numpy'

In [33]:
np.asarray(s)

array([-1.97752349,  0.18526196, -1.04453834,  0.7896768 , -0.50487928])

When the Series or Index is backed by an ExtensionArray, to_numpy() may involve copying data and coercing values.
__to_numpy() gives some control over the dtype of the resulting numpy.ndarray.__ For example, consider datetimes with timezones. __NumPy doesn’t have a dtype to represent timezone-aware datetimes__, so there are two possibly useful representations:

1. An __object-dtype numpy.ndarray with Timestamp objects__, each with the correct tz
2. A __datetime64[ns] -dtype numpy.ndarray__, where the values have been converted to UTC and the timezone discarded

#### Timezones may be preserved with dtype=object

In [34]:
ser = pd.Series(pd.date_range('2000', periods=2, tz="CET"))
ser

0   2000-01-01 00:00:00+01:00
1   2000-01-02 00:00:00+01:00
dtype: datetime64[ns, CET]

In [35]:
pd.Series(pd.date_range('2000', periods=2, tz="UTC"))

0   2000-01-01 00:00:00+00:00
1   2000-01-02 00:00:00+00:00
dtype: datetime64[ns, UTC]

In [39]:
ser.to_numpy(dtype=object)

AttributeError: 'Series' object has no attribute 'to_numpy'

array([Timestamp('2000-01-01 00:00:00+0100', tz='CET', freq='D'),
       Timestamp('2000-01-02 00:00:00+0100', tz='CET', freq='D')],
      dtype=object)

In [38]:
np.asarray(ser, dtype=object)

array([946681200000000000, 946767600000000000], dtype=object)

#### Or thrown away with dtype='datetime64[ns]'

In [ ]:
ser.to_numpy(dtype="datetime64[ns]")

In [42]:
np.asarray(ser, dtype='datetime64[ns]')

array(['1999-12-31T23:00:00.000000000', '2000-01-01T23:00:00.000000000'],
      dtype='datetime64[ns]')

Getting the “raw data” inside a DataFrame is possibly a bit more complex. When your DataFrame only has a single data type for all the columns, DataFrame.to_numpy() will return the underlying data:

In [ ]:
df.to_numpy()

array([[-0.1732,  0.1192, -1.0442],
       [-0.8618, -2.1046, -0.4949],
       [ 1.0718,  0.7216, -0.7068],
       [-1.0396,  0.2719, -0.425 ],
       [ 0.567 ,  0.2762, -1.0874],
       [-0.6737,  0.1136, -1.4784],
       [ 0.525 ,  0.4047,  0.577 ],
       [-1.715 , -1.0393, -0.3706]])

__If a DataFrame contains homogeneously-typed data, the ndarray can actually be modified in-place, and the changes will be reflected in the data structure__. For heterogeneous data (e.g. some of the DataFrame’s columns are not all the same dtype), this will not be the case. The values attribute itself, unlike the axis labels, cannot be assigned to.

Note: __When working with heterogeneous data, the dtype of the resulting ndarray will be chosen to accommodate all of the data involved__. For example, if strings are involved, the result will be of object dtype. If there are only floats and integers, the resulting array will be of float dtype.

In the past, pandas recommended Series.values or DataFrame.values for extracting the data from a Series or DataFrame. You’ll still find references to these in old code bases and online. Going forward, __we recommend avoiding .values and using .array or .to_numpy(). .values has the following drawbacks:__

#### 1. When your Series contains an extension type, it’s unclear whether Series.values returns a NumPy array or the extension array. Series.array will always return an ExtensionArray, and will never copy data. Series.to_numpy() will always return a NumPy array, potentially at the cost of copying / coercing values.
#### 2. When your DataFrame contains a mixture of data types, DataFrame.values may involve copying data and coercing values to a common dtype, a relatively expensive operation. DataFrame.to_numpy(), being a method, makes it clearer that the returned NumPy array may not be a view on the same data in the DataFrame.

## Accelerated operations

pandas has support for accelerating certain types of binary numerical and boolean operations using the __numexpr__ library and the __bottleneck__ libraries.

These libraries are especially useful when __dealing with large data sets, and provide large speedups. numexpr uses smart chunking, caching, and multiple cores. bottleneck is a set of specialized cython routines that are especially fast when dealing with arrays that have nans.__

Here is a sample (using 100 column x 100,000 row DataFrames):

Operation	0.11.0 (ms)	Prior Version (ms)	Ratio to Prior
df1 > df2	13.32	125.35	0.1063
df1 * df2	21.71	36.63	0.5928
df1 + df2	22.04	36.50	0.6039
You are highly encouraged to install both libraries. See the section Recommended Dependencies for more installation info.

__These are both enabled to be used by default, you can control this by setting the options:__

New in version 0.20.0.

pd.set_option('compute.use_bottleneck', False)
pd.set_option('compute.use_numexpr', False)

## Flexible binary operations

With binary operations between pandas data structures, there are two key points of interest:

1. __Broadcasting behavior between higher- (e.g. DataFrame) and lower-dimensional (e.g. Series) objects.__
2. __Missing data in computations.__

We will demonstrate how to manage these issues independently, though they can be handled simultaneously.

### Matching / broadcasting behavior

DataFrame has the methods __add(), sub(), mul(), div()__ and related functions __radd(), rsub(), … for carrying out binary operations__. For broadcasting behavior, Series input is of primary interest. Using these functions, you can use to either match on the index or columns via the __axis__ keyword:

In [43]:
df = pd.DataFrame({
        'one': pd.Series(np.random.randn(3), index=['a', 'b', 'c']),
        'two': pd.Series(np.random.randn(4), index=['a', 'b', 'c', 'd']),
        'three': pd.Series(np.random.randn(3), index=['b', 'c', 'd'])})

In [44]:
df

,one,two,three
a,0.864781,-1.202497,NaN
b,-0.505994,0.608828,-0.530912
c,-0.939319,0.433543,1.357191
d,NaN,-0.762292,-0.916487


In [98]:
row = df.iloc[1]
row

one     -0.505994
two      0.608828
three   -0.530912
Name: b, dtype: float64

In [100]:
row.dtype

dtype('float64')

In [101]:
type(row)

pandas.core.series.Series

In [102]:
column = df['two']
column

a   -1.202497
b    0.608828
c    0.433543
d   -0.762292
Name: two, dtype: float64

In [103]:
type(column)

pandas.core.series.Series

In [48]:
df.sub(row, axis='columns') # CROSS COLUMNS

,one,two,three
a,1.370775,-1.811325,NaN
b,0.000000,0.000000,0.000000
c,-0.433326,-0.175285,1.888103
d,NaN,-1.371120,-0.385574


In [49]:
df.sub(row, axis=1) # CROSS COLUMNS

,one,two,three
a,1.370775,-1.811325,NaN
b,0.000000,0.000000,0.000000
c,-0.433326,-0.175285,1.888103
d,NaN,-1.371120,-0.385574


In [50]:
df.sub(column, axis='index') # CROSS ROWS

,one,two,three
a,2.067278,0.0,NaN
b,-1.114822,0.0,-1.139740
c,-1.372863,0.0,0.923648
d,NaN,0.0,-0.154195


In [52]:
df.sub(column, axis=0) # CROSS ROWS

,one,two,three
a,2.067278,0.0,NaN
b,-1.114822,0.0,-1.139740
c,-1.372863,0.0,0.923648
d,NaN,0.0,-0.154195


#### Furthermore you can align a level of a MultiIndexed DataFrame with a Series.

In [64]:
dfmi = df.copy()
dfmi

,one,two,three
a,0.864781,-1.202497,NaN
b,-0.505994,0.608828,-0.530912
c,-0.939319,0.433543,1.357191
d,NaN,-0.762292,-0.916487


In [65]:
dfmi.index = pd.MultiIndex.from_tuples([(1, 'a'), (1, 'b'), (1, 'c'), (2, 'a')], names=['first', 'second'])

In [66]:
dfmi

one       two     three
first second                              
1     a       0.864781 -1.202497       NaN
      b      -0.505994  0.608828 -0.530912
      c      -0.939319  0.433543  1.357191
2     a            NaN -0.762292 -0.916487

In [56]:
dfmi.sub(column, axis=0, level='second')

one       two     three
first second                              
1     a       2.067278  0.000000       NaN
      b      -1.114822  0.000000 -1.139740
      c      -1.372863  0.000000  0.923648
2     a            NaN  0.440205  0.286011

In [59]:
dfmi.sub(column, axis=0, level='first')

one  two  three
first second                 
1     a       NaN  NaN    NaN
      b       NaN  NaN    NaN
      c       NaN  NaN    NaN
2     a       NaN  NaN    NaN

In [61]:
dfmi.sub(row, axis=1, level='second')

one       two     three
first second                              
1     a       1.370775 -1.811325       NaN
      b       0.000000  0.000000  0.000000
      c      -0.433326 -0.175285  1.888103
2     a            NaN -1.371120 -0.385574

In [63]:
dfmi.sub(row, axis=1, level='first')

one       two     three
first second                              
1     a       1.370775 -1.811325       NaN
      b       0.000000  0.000000  0.000000
      c      -0.433326 -0.175285  1.888103
2     a            NaN -1.371120 -0.385574

dfmi.sub(column, axis=0) - ValueError: cannot join with no level specified and no overlapping names

In [69]:
dfmi.sub(row, axis=0, level='second')

one  two  three
first second                 
1     a       NaN  NaN    NaN
      b       NaN  NaN    NaN
      c       NaN  NaN    NaN
2     a       NaN  NaN    NaN

Series and Index also support the divmod() builtin. This function takes the floor division and modulo operation at the same time returning a two-tuple of the same type as the left hand side. For example:

In [79]:
s = pd.Series(np.arange(10))
s

0    0
1    1
2    2
3    3
4    4
5    5
6    6
7    7
8    8
9    9
dtype: int32

In [74]:
div ,reminder = divmod(s, 3)

In [75]:
div

0    0
1    0
2    0
3    1
4    1
5    1
6    2
7    2
8    2
9    3
dtype: int32

In [76]:
reminder

0    0
1    1
2    2
3    0
4    1
5    2
6    0
7    1
8    2
9    0
dtype: int32

In [77]:
idx = pd.Index(np.arange(10))

In [78]:
idx

Int64Index([0, 1, 2, 3, 4, 5, 6, 7, 8, 9], dtype='int64')

In [80]:
div, rem = divmod(idx, 3)

In [81]:
div

Int64Index([0, 0, 0, 1, 1, 1, 2, 2, 2, 3], dtype='int64')

In [82]:
rem

Int64Index([0, 1, 2, 0, 1, 2, 0, 1, 2, 0], dtype='int64')

##### We can also do elementwise divmod():

In [83]:
div, rem = divmod(s, [2, 2, 3, 3, 4, 4, 5, 5, 6, 6])

In [84]:
div

0    0
1    0
2    0
3    1
4    1
5    1
6    1
7    1
8    1
9    1
dtype: int32

In [85]:
rem

0    0
1    1
2    2
3    0
4    0
5    1
6    1
7    2
8    2
9    3
dtype: int32

### Missing data / operations with fill values

__In Series and DataFrame, the arithmetic functions have the option of inputting a fill_value__, namely a value to substitute when at most one of the values at a location are missing. For example, when adding two DataFrame objects, __you may wish to treat NaN as 0 unless both DataFrames are missing that value__, in which case the result will be NaN (you can later replace NaN with some other value using fillna if you wish).

In [86]:
df

,one,two,three
a,0.864781,-1.202497,NaN
b,-0.505994,0.608828,-0.530912
c,-0.939319,0.433543,1.357191
d,NaN,-0.762292,-0.916487


In [107]:
df2 = df.copy()
#df2.iloc[0, 'three'] #ValueError: Location based indexing can only have [integer, integer slice (START point is INCLUDED, END point is EXCLUDED), listlike of integers, boolean array] types
df2.iloc[0, 2] = 3.456

In [108]:
df2

,one,two,three
a,0.864781,-1.202497,3.456000
b,-0.505994,0.608828,-0.530912
c,-0.939319,0.433543,1.357191
d,NaN,-0.762292,-0.916487


In [96]:
df + df2

,one,two,three
a,1.729562,-2.404994,NaN
b,-1.011988,1.217656,-1.061824
c,-1.878639,0.867086,2.714382
d,NaN,-1.524584,-1.832973


In [97]:
df.add(df2, fill_value=0)

,one,two,three
a,1.729562,-2.404994,3.456000
b,-1.011988,1.217656,-1.061824
c,-1.878639,0.867086,2.714382
d,NaN,-1.524584,-1.832973


### Flexible comparisons

Series and DataFrame have the __binary comparison methods eq, ne, lt, gt, le, and ge__ whose behavior is analogous to the binary arithmetic operations described above:

#### NAN value atleast in one data frame lead to False on all comparison methods except ne

These operations __produce a pandas object of the same type as the left-hand-side input that is of dtype bool__. These boolean objects can be used in indexing operations, see the section on Boolean indexing.

In [109]:
df.gt(df2)

,one,two,three
a,False,False,False
b,False,False,False
c,False,False,False
d,False,False,False


In [111]:
df.lt(df2)

,one,two,three
a,False,False,False
b,False,False,False
c,False,False,False
d,False,False,False


In [112]:
df.eq(df2)

,one,two,three
a,True,True,False
b,True,True,True
c,True,True,True
d,False,True,True


In [106]:
df2.ne(df)

,one,two,three
a,False,False,True
b,False,False,False
c,False,False,False
d,True,False,False


### Boolean reductions

ou can apply the reductions: __empty, any(), all(), and bool()__ to provide a way to summarize a boolean result.

In [114]:
(df > 0).all()

one      False
two      False
three    False
dtype: bool

In [115]:
(df > 0).any()

one      True
two      True
three    True
dtype: bool

You can reduce to a final boolean value.

In [116]:
(df > 0).any().any()

True

In [117]:
df.empty

False

In [118]:
pd.DataFrame(columns=['A']).empty

True

#### To evaluate single-element pandas objects in a boolean context, use the method bool():

In [119]:
pd.Series([True]).bool()

True

In [120]:
pd.Series([False]).bool()

False

In [121]:
pd.DataFrame([[True]]).bool()

True

In [124]:
pd.DataFrame([[False]]).bool()

False

### Comparing if objects are equivalent

#### (df + df == df * 2).all(). But in fact, this expression is False:

In [126]:
df + df == df * 2 # NAA effect

,one,two,three
a,True,True,False
b,True,True,True
c,True,True,True
d,False,True,True


In [127]:
(df + df == df * 2).all()

one      False
two       True
three    False
dtype: bool

Notice that the boolean DataFrame df + df == df * 2 contains some False values! This is because NaNs do not compare as equals:

In [128]:
np.nan == np.nan

False

equality method with NaNs in corresponding locations treated as equal.

In [129]:
(df + df).equals(df * 2)

True

#### Note that the Series or DataFrame index needs to be in the same order for equality to be True:

In [130]:
df1 = pd.DataFrame({'col': ['foo', 0, np.nan]})

In [131]:
df2 = pd.DataFrame({'col': [np.nan, 0, 'foo']}, index=[2, 1, 0])

In [132]:
df1

,col
0,foo
1,0
2,NaN


In [133]:
df2

,col
2,NaN
1,0
0,foo


In [134]:
df1.equals(df2)

False

In [135]:
df1.equals(df2.sort_index())

True

### Comparing array-like objects

You can conveniently perform element-wise comparisons when __comparing a pandas data structure with a scalar value__:

In [136]:
pd.Series(['foo', 'bar', 'baz']) == 'foo'

0     True
1    False
2    False
dtype: bool

In [137]:
pd.Index(['foo', 'bar', 'baz']) == 'foo'

array([ True, False, False])

Pandas also handles __element-wise comparisons__ between different array-like __objects of the same length__:

In [138]:
pd.Series(['foo', 'bar', 'baz']) == pd.Index(['foo', 'bar', 'qux'])

0     True
1     True
2    False
dtype: bool

In [139]:
pd.Series(['foo', 'bar', 'baz']) == np.array(['foo', 'bar', 'qux'])

0     True
1     True
2    False
dtype: bool

Trying to compare Index or Series objects of __different lengths will raise a ValueError:__

pd.Series(['foo', 'bar', 'baz']) == pd.Series(['foo', 'bar']) - ValueError: Series lengths must match to compare

pd.Series(['foo', 'bar', 'baz']) == pd.Series(['foo']) - ValueError: Series lengths must match to compare

#### Note that this is different from the NumPy behavior where a comparison can be broadcast:

In [141]:
np.array([1, 2, 3]) == np.array([2])

array([False,  True, False])

or it can return False if broadcasting can not be done:

In [142]:
np.array([1, 2, 3]) == np.array([1, 2])

E:\Study\ML\Anaconda\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: elementwise == comparison failed; this will raise an error in the future.
  """Entry point for launching an IPython kernel.


False

### Combining overlapping data sets

A problem occasionally arising is the combination of two similar data sets where __values in one are preferred over the other__. An example would be two data series representing a particular economic indicator where one is considered to be of “higher quality”. However, the lower quality series might extend further back in history or have more complete data coverage. As such, we would like to combine two DataFrame objects where __missing values in one DataFrame are conditionally filled with like-labeled values from the other DataFrame__. The function implementing this operation is ___combine_first()__, which we illustrate:

In [146]:
df1 = pd.DataFrame({'A': [1., np.nan, 3., 5., np.nan], 'B': [np.nan, 2., 3., np.nan, 6.]})
df2 = pd.DataFrame({'A': [5., 2., 4., np.nan, 3., 7.], 'B': [np.nan, np.nan, 3., 4., 6., 8.]})

In [147]:
df1

,A,B
0,1.0,NaN
1,NaN,2.0
2,3.0,3.0
3,5.0,NaN
4,NaN,6.0


In [148]:
df2

,A,B
0,5.0,NaN
1,2.0,NaN
2,4.0,3.0
3,NaN,4.0
4,3.0,6.0
5,7.0,8.0


In [149]:
df1.combine_first(df2)

,A,B
0,1.0,NaN
1,2.0,2.0
2,3.0,3.0
3,5.0,4.0
4,3.0,6.0
5,7.0,8.0


In [150]:
df2

,A,B
0,5.0,NaN
1,2.0,NaN
2,4.0,3.0
3,NaN,4.0
4,3.0,6.0
5,7.0,8.0


In [153]:
df1

,A,B
0,1.0,NaN
1,NaN,2.0
2,3.0,3.0
3,5.0,NaN
4,NaN,6.0


### DataFrame.combine_first(self, other)
Update null elements with value in the same location in other.

Combine two DataFrame objects by filling null values in one DataFrame with non-null values from other DataFrame. The row and column indexes of the resulting DataFrame will be the union of the two.

In [154]:
df1 = pd.DataFrame({'A': [None, 0], 'B': [None, 4]})
df2 = pd.DataFrame({'A': [1, 1], 'B': [3, 3]})
df1.combine_first(df2)

,A,B
0,1.0,3.0
1,0.0,4.0


Null values still persist if the location of that null value does not exist in other

In [155]:
df1 = pd.DataFrame({'A': [None, 0], 'B': [4, None]})
df2 = pd.DataFrame({'B': [3, 3], 'C': [1, 1]}, index=[1, 2])
df1.combine_first(df2)

,A,B,C
0,NaN,4.0,NaN
1,0.0,3.0,1.0
2,NaN,3.0,1.0


### General DataFrame combine

The combine_first() method above calls the more general DataFrame.combine(). This method takes another DataFrame and a combiner function, aligns the input DataFrame and then passes the combiner function pairs of Series (i.e., columns whose names are the same).

So, for instance, to reproduce combine_first() as above:

In [152]:
def combiner(x, y):
     return np.where(pd.isna(x), y, x)

### DataFrame.combine(self, other, func, fill_value=None, overwrite=True)

Perform column-wise combine with another DataFrame.

Combines a DataFrame with other DataFrame using func to element-wise combine columns. The row and column indexes of the resulting DataFrame will be the union of the two.

#### Parameters:
fill_value : scalar value, default None - The value to fill NaNs with prior to passing any column to the merge func.

overwrite : bool, default True - If True, columns in self that do not exist in other will be overwritten with NaNs.

Combine using a simple function that chooses the smaller column.

In [157]:
df1 = pd.DataFrame({'A': [0, 0], 'B': [4, 4]})
df2 = pd.DataFrame({'A': [1, 1], 'B': [3, 3]})
take_smaller = lambda s1, s2: s1 if s1.sum() < s2.sum() else s2
df1.combine(df2, take_smaller)

,A,B
0,0,3
1,0,3


Example using a true element-wise combine function.

In [160]:
df1 = pd.DataFrame({'A': [5, 0], 'B': [2, 4]})
df2 = pd.DataFrame({'A': [1, 1], 'B': [3, 3]})
df1.combine(df2, np.minimum)

,A,B
0,1,2
1,0,3


Using fill_value fills Nones prior to passing the column to the merge function.

In [161]:
df1 = pd.DataFrame({'A': [0, 0], 'B': [None, 4]})
df2 = pd.DataFrame({'A': [1, 1], 'B': [3, 3]})
df1.combine(df2, take_smaller, fill_value=-5)

,A,B
0,0,-5.0
1,0,4.0


However, if the same element in both dataframes is None, that None is preserved

In [162]:
df1 = pd.DataFrame({'A': [0, 0], 'B': [None, 4]})
df2 = pd.DataFrame({'A': [1, 1], 'B': [None, 3]})
df1.combine(df2, take_smaller, fill_value=-5)

,A,B
0,0,-5.0
1,0,3.0


Example that demonstrates the use of overwrite and behavior when the axis differ between the dataframes.

In [163]:
df1 = pd.DataFrame({'A': [0, 0], 'B': [4, 4]})
df2 = pd.DataFrame({'B': [3, 3], 'C': [-10, 1], }, index=[1, 2])
df1.combine(df2, take_smaller)

,A,B,C
0,NaN,NaN,NaN
1,NaN,3.0,-10.0
2,NaN,3.0,1.0


In [164]:
df1.combine(df2, take_smaller, overwrite=False)

,A,B,C
0,0.0,NaN,NaN
1,0.0,3.0,-10.0
2,NaN,3.0,1.0


In [165]:
df2 = pd.DataFrame({'B': [3, 3], 'C': [1, 1], }, index=[1, 2])
df2.combine(df1, take_smaller)

,A,B,C
0,0.0,NaN,NaN
1,0.0,3.0,NaN
2,NaN,3.0,NaN


In [166]:
df2.combine(df1, take_smaller, overwrite=False)

,A,B,C
0,0.0,NaN,NaN
1,0.0,3.0,1.0
2,NaN,3.0,1.0


## Descriptive statistics

There exists a large number of methods for computing descriptive statistics and other related operations on Series, DataFrame. Most of these are __aggregations (hence producing a lower-dimensional result) like sum(), mean(), and quantile()__, but some of them, __like cumsum() and cumprod(), produce an object of the same size__. Generally speaking, these methods take an __axis argument, just like ndarray.{sum, std, …}, but the axis can be specified by name or integer:__

Series: no axis argument needed

DataFrame: “index” (axis=0, default), “columns” (axis=1)

In [167]:
df

,one,two,three
a,0.864781,-1.202497,NaN
b,-0.505994,0.608828,-0.530912
c,-0.939319,0.433543,1.357191
d,NaN,-0.762292,-0.916487


In [168]:
df.mean(0)

one     -0.193511
two     -0.230604
three   -0.030069
dtype: float64

In [169]:
df.mean(1)

a   -0.168858
b   -0.142693
c    0.283805
d   -0.839389
dtype: float64

All such methods have a __skipna__ option signaling whether to exclude missing data (True by default):

In [170]:
df.sum(0, skipna=False)

one           NaN
two     -0.922418
three         NaN
dtype: float64

In [172]:
df.sum(axis=1, skipna=True)

a   -0.337716
b   -0.428078
c    0.851415
d   -1.678779
dtype: float64

In [173]:
df.std()

one      0.941769
two      0.889383
three    1.216773
dtype: float64

Combined with the broadcasting / arithmetic behavior, one can describe __various statistical procedures, like standardization__ (rendering data zero mean and standard deviation 1), very concisely:

In [175]:
ts_stand = (df - df.mean()) / df.std()
ts_stand

,one,two,three
a,1.123728,-1.092772,NaN
b,-0.331804,0.943837,-0.411616
c,-0.791923,0.746751,1.140115
d,NaN,-0.597816,-0.728499


In [176]:
ts_stand.std()

one      1.0
two      1.0
three    1.0
dtype: float64

In [177]:
xs_std = df.sub(df.mean(1), axis=0).div(df.std(1), axis=0)

In [178]:
xs_std

,one,two,three
a,0.707107,-0.707107,NaN
b,-0.558105,1.154489,-0.596384
c,-1.058474,0.129581,0.928893
d,NaN,0.707107,-0.707107


In [180]:
xs_std2 = df.sub(df.mean(1), axis=1).div(df.std(1), axis=1)

In [181]:
xs_std2

,a,b,c,d,one,three,two
a,NaN,NaN,NaN,NaN,NaN,NaN,NaN
b,NaN,NaN,NaN,NaN,NaN,NaN,NaN
c,NaN,NaN,NaN,NaN,NaN,NaN,NaN
d,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [182]:
xs_std3 = df.sub(df.mean(0), axis=1).div(df.std(0), axis=1)

In [183]:
xs_std3

,one,two,three
a,1.123728,-1.092772,NaN
b,-0.331804,0.943837,-0.411616
c,-0.791923,0.746751,1.140115
d,NaN,-0.597816,-0.728499


In [185]:
df.sub(df.mean(0), axis=0).div(df.std(0), axis=0)

,one,two,three
a,NaN,NaN,NaN
b,NaN,NaN,NaN
c,NaN,NaN,NaN
d,NaN,NaN,NaN
one,NaN,NaN,NaN
three,NaN,NaN,NaN
two,NaN,NaN,NaN


#### Note that methods like cumsum() and cumprod() preserve the location of NaN values. This is somewhat different from expanding() and rolling()

In [191]:
df.cumsum()

,one,two,three
a,0.864781,-1.202497,NaN
b,0.358787,-0.593669,-0.530912
c,-0.580532,-0.160126,0.826279
d,NaN,-0.922418,-0.090208


In [192]:
df

,one,two,three
a,0.864781,-1.202497,NaN
b,-0.505994,0.608828,-0.530912
c,-0.939319,0.433543,1.357191
d,NaN,-0.762292,-0.916487


In [187]:
df.expanding()

Expanding [min_periods=1,center=False,axis=0]

In [190]:
df.expanding(1).sum()

,one,two,three
a,0.864781,-1.202497,NaN
b,0.358787,-0.593669,-0.530912
c,-0.580532,-0.160126,0.826279
d,-0.580532,-0.922418,-0.090208


In [193]:
df.expanding(2).sum()

,one,two,three
a,NaN,NaN,NaN
b,0.358787,-0.593669,NaN
c,-0.580532,-0.160126,0.826279
d,-0.580532,-0.922418,-0.090208


Here is a quick reference summary table of common functions. Each also takes an optional __level parameter which applies only if the object has a hierarchical index.__

#### Function	- Description
#### count	- Number of non-NA observations
#### sum	- Sum of values
#### mean	- Mean of values
#### mad	- Mean absolute deviation
#### median- 	Arithmetic median of values
#### min	- Minimum
#### max	- Maximum
#### mode	- Mode
#### abs	- Absolute Value
#### prod	- Product of values
#### std	- Bessel-corrected sample standard deviation
#### var	- Unbiased variance
#### sem	- Standard error of the mean
#### skew	- Sample skewness (3rd moment)
#### kurt	- Sample kurtosis (4th moment)
#### quantile	- Sample quantile (value at %)
#### cumsum	- Cumulative sum
#### cumprod	- Cumulative product
#### cummax	- Cumulative maximum
#### cummin	 - Cumulative minimum#### 

#### Note that by chance some NumPy methods, like mean, std, and sum, will exclude NAs on Series input by default:

In [196]:
np.mean(df['one'])

-0.1935107412332541

In [197]:
#np.mean(df['one'].to_numpy())
np.mean(np.asarray(df['one']))

nan

__Series.nunique()__ will return the number of unique non-NA values in a Series:

In [199]:
series = pd.Series(np.random.randn(500))
series[20:500] = np.nan
series[10:20] = 5

In [200]:
series.nunique()

11

### Summarizing data: describe

There is a convenient describe() function which computes a variety of summary statistics about a Series or the columns of a DataFrame (excluding NAs of course):

In [202]:
series = pd.Series(np.random.randn(100))

In [203]:
series[::2] = np.nan

In [204]:
series.describe()

count    50.000000
mean     -0.020881
std       0.949686
min      -2.614120
25%      -0.488005
50%       0.037697
75%       0.672136
max       1.716057
dtype: float64

In [205]:
frame = pd.DataFrame(np.random.randn(1000, 5), columns=['a', 'b', 'c', 'd', 'e'])
frame.iloc[::2] = np.nan

In [206]:
frame.describe()

,a,b,c,d,e
count,500.000000,500.000000,500.000000,500.000000,500.000000
mean,0.002926,-0.018767,-0.037774,-0.013924,0.062222
std,0.972788,1.032731,1.029517,1.070016,0.938438
min,-2.735316,-3.460359,-2.967345,-3.013532,-3.535795
25%,-0.620892,-0.769382,-0.789362,-0.756785,-0.546375
50%,-0.016092,-0.010540,0.000462,-0.082319,0.051071
75%,0.685322,0.750672,0.660575,0.726910,0.670640
max,2.569829,2.676299,2.903974,2.791521,3.286513


You can select __specific percentiles__ to include in the output:

In [207]:
series.describe(percentiles=[.05, .25, .75, .95])

count    50.000000
mean     -0.020881
std       0.949686
min      -2.614120
5%       -1.613299
25%      -0.488005
50%       0.037697
75%       0.672136
95%       1.414291
max       1.716057
dtype: float64

For a __non-numerical Series__ object, describe() will give a __simple summary of the number of unique values and most frequently occurring values:__

In [210]:
s = pd.Series(['a', 'a', 'b', 'b', 'a', 'a', np.nan, 'c', 'd', 'a'])

In [209]:
s.describe()

count     9
unique    4
top       a
freq      5
dtype: object

Note that on a __mixed-type DataFrame object__, describe() will restrict the summary to include __only numerical columns or, if none are, only categorical columns:__

In [212]:
frame = pd.DataFrame({'a': ['Yes', 'Yes', 'No', 'No'], 'b': range(4)})
frame.describe()

,b
count,4.000000
mean,1.500000
std,1.290994
min,0.000000
25%,0.750000
50%,1.500000
75%,2.250000
max,3.000000


This behavior can be controlled by providing a list of types as __include/exclude__ arguments. The special value __all__ can also be used:

In [214]:
frame.describe(include=['object'])

,a
count,4
unique,2
top,Yes
freq,2


In [215]:
frame.describe(include=['number'])

,b
count,4.000000
mean,1.500000
std,1.290994
min,0.000000
25%,0.750000
50%,1.500000
75%,2.250000
max,3.000000


In [217]:
#frame.describe(include=['all']) - TypeError: data type "all" not understood
frame.describe(include='all')

,a,b
count,4,4.000000
unique,2,NaN
top,Yes,NaN
freq,2,NaN
mean,NaN,1.500000
std,NaN,1.290994
min,NaN,0.000000
25%,NaN,0.750000
50%,NaN,1.500000
75%,NaN,2.250000


That feature relies on __select_dtypes__ method

### Index of min/max values

In [218]:
s1 = pd.Series(np.random.randn(5))

In [219]:
s1.idxmax(), s1.idxmin()

(4, 1)

In [220]:
df1 = pd.DataFrame(np.random.randn(5, 3), columns=['A', 'B', 'C'])
df1

,A,B,C
0,-0.534448,-1.401290,-1.819716
1,0.578691,-1.749486,1.101898
2,-1.287205,1.185475,-2.753929
3,1.107121,0.924748,1.329205
4,0.032745,-0.435888,-0.620921


In [221]:
df1.idxmax(axis=0)

A    3
B    2
C    3
dtype: int64

In [222]:
df1.idxmin(axis=1)

0    C
1    B
2    C
3    B
4    C
dtype: object

When there are multiple rows (or columns) matching the minimum or maximum value, __idxmin() and idxmax() return the first matching index:__

In [223]:
df3 = pd.DataFrame([2, 1, 1, 3, np.nan], columns=['A'], index=list('edcba'))

In [224]:
df3

,A
e,2.0
d,1.0
c,1.0
b,3.0
a,NaN


In [226]:
df3.idxmin()

A    d
dtype: object

#### Note idxmin and idxmax are called argmin and argmax in NumPy.

### Value counts (histogramming) / mode

The __value_counts()__ Series method and top-level function __computes a histogram of a 1D array of values__. It can also be used as a function on regular arrays:

In [228]:
data = np.random.randint(0, 7, size=50)

In [229]:
data

array([6, 3, 2, 3, 1, 2, 3, 2, 5, 3, 4, 0, 4, 6, 0, 2, 4, 4, 1, 1, 0, 3,
       5, 2, 4, 2, 6, 3, 3, 0, 0, 5, 6, 0, 6, 5, 2, 5, 2, 2, 1, 3, 1, 5,
       1, 5, 0, 2, 2, 1])

In [230]:
s = pd.Series(data)

In [231]:
s.value_counts()

2    11
3     8
5     7
1     7
0     7
6     5
4     5
dtype: int64

In [232]:
pd.value_counts(data)

2    11
3     8
5     7
1     7
0     7
6     5
4     5
dtype: int64

Similarly, you can get the most frequently occurring value(s) (__the mode__) of the values in a Series or DataFrame:

In [233]:
s5 = pd.Series([1, 1, 3, 3, 3, 5, 5, 7, 7, 7])

In [234]:
s5.mode()

0    3
1    7
dtype: int64

In [235]:
df5 = pd.DataFrame({"A": np.random.randint(0, 7, size=50),  "B": np.random.randint(-10, 15, size=50)})

In [236]:
df5.mode()

,A,B
0,0,-10


### Discretization and quantiling

#### Continuous values can be discretized using the cut() (bins based on values) and qcut() (bins based on sample quantiles) functions:

In [243]:
arr = np.random.randn(10)
arr

array([-1.31511447, -0.54804975, -0.48920087, -0.02411389, -0.82844197,
        0.87213778, -0.25398622,  1.55317079,  0.18984647,  0.37694103])

In [244]:
factor = pd.cut(arr, 4)

In [245]:
factor

[(-1.318, -0.598], (-0.598, 0.119], (-0.598, 0.119], (-0.598, 0.119], (-1.318, -0.598], (0.836, 1.553], (-0.598, 0.119], (0.836, 1.553], (0.119, 0.836], (0.119, 0.836]]
Categories (4, interval[float64]): [(-1.318, -0.598] < (-0.598, 0.119] < (0.119, 0.836] < (0.836, 1.553]]

In [246]:
factor = pd.cut(arr, [-5, -1, 0, 1, 5])

In [247]:
factor

[(-5, -1], (-1, 0], (-1, 0], (-1, 0], (-1, 0], (0, 1], (-1, 0], (1, 5], (0, 1], (0, 1]]
Categories (4, interval[int64]): [(-5, -1] < (-1, 0] < (0, 1] < (1, 5]]

__qcut() computes sample quantiles.__ For example, we could slice up some normally distributed data __into equal-size quartiles like so:__

In [249]:
arr = np.random.randn(30)
factor = pd.qcut(arr, [0, .25, .5, .75, 1])

In [250]:
factor

[(-0.714, 0.0485], (-1.5459999999999998, -0.714], (0.93, 2.668], (-0.714, 0.0485], (-1.5459999999999998, -0.714], ..., (-1.5459999999999998, -0.714], (-0.714, 0.0485], (-1.5459999999999998, -0.714], (-0.714, 0.0485], (0.93, 2.668]]
Length: 30
Categories (4, interval[float64]): [(-1.5459999999999998, -0.714] < (-0.714, 0.0485] < (0.0485, 0.93] < (0.93, 2.668]]

In [251]:
pd.value_counts(factor)

(0.93, 2.668]                    8
(-1.5459999999999998, -0.714]    8
(0.0485, 0.93]                   7
(-0.714, 0.0485]                 7
dtype: int64

We can also __pass infinite values__ to define the bins:

In [253]:
arr = np.random.randn(20)

In [254]:
factor = pd.cut(arr, [-np.inf, 0, np.inf])

In [255]:
factor

[(-inf, 0.0], (-inf, 0.0], (-inf, 0.0], (-inf, 0.0], (-inf, 0.0], ..., (-inf, 0.0], (-inf, 0.0], (0.0, inf], (-inf, 0.0], (0.0, inf]]
Length: 20
Categories (2, interval[float64]): [(-inf, 0.0] < (0.0, inf]]

## Function application

To apply your own or another library’s functions to pandas objects, you should be aware of the three methods below. The appropriate method to use depends on whether your function expects to operate on an entire DataFrame or Series, row- or column-wise, or elementwise.

1. Tablewise Function Application: pipe()
2. Row or Column-wise Function Application: apply()
3. Aggregation API: agg() and transform()
4. Applying Elementwise Functions: applymap()

### Tablewise function application

DataFrames and Series can of course just be passed into functions. However, if the function __needs to be called in a chain, consider using the pipe()__ method. Compare the following

##### f, g, and h are functions taking and returning ``DataFrames``
>f(g(h(df), arg1=1), arg2=2, arg3=3)

with the equivalent

>(df.pipe(h).pipe(g, arg1=1).pipe(f, arg2=2, arg3=3))

pipe makes it easy to use your own or another library’s functions in method chains, alongside pandas’ methods.

In the example above, the functions f, g, and h each expected the DataFrame as the first positional argument. What if the function you wish to apply takes its data as, say, the second argument? In this case, provide __pipe with a tuple of (callable, data_keyword)__. .pipe will route the DataFrame to the argument specified in the tuple.

For example, we can fit a regression using statsmodels. Their API expects a formula first and a DataFrame as the second argument, data. We pass in the function, keyword pair (sm.ols, 'data') to pipe:

In [2]:
import statsmodels.formula.api as sm

bb = pd.read_csv('baseball.csv', index_col='id')

(bb.query('h > 0')
         .assign(ln_h = lambda df:np.log(df.h))
         .pipe((sm.ols, 'data'), 'hr ~ ln_h + year + g + C(lg)')
         .fit()
         .summary()
)
         

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                     hr   R-squared:                       0.685
Model:                            OLS   Adj. R-squared:                  0.665
Method:                 Least Squares   F-statistic:                     34.28
Date:                Thu, 15 Aug 2019   Prob (F-statistic):           3.48e-15
Time:                        23:18:21   Log-Likelihood:                -205.92
No. Observations:                  68   AIC:                             421.8
Df Residuals:                      63   BIC:                             432.9
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
===============================================================================
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
Intercept   -8484.7720   4664.146     -1.819      0.074   -1.78e+04     835.780
C(lg)[T.NL]    -2.2736      1.325     -1.716      0.091      -4.922       0.375
ln_h           -1.3542      0.875     -1.547      0.127      -3.103       0.395
year            4.2277      2.324      1.819      0.074      -0.417       8.872
g               0.1841      0.029      6.258      0.000       0.125       0.243
==============================================================================
Omnibus:                       10.875   Durbin-Watson:                   1.999
Prob(Omnibus):                  0.004   Jarque-Bera (JB):               17.298
Skew:                           0.537   Prob(JB):                     0.000175
Kurtosis:                       5.225   Cond. No.                     1.49e+07
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.49e+07. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

### Row or column-wise function application

__Arbitrary functions can be applied along the axes of a DataFrame using the apply() method__, which, like the descriptive statistics methods, takes an optional axis argument:

In [4]:
df = pd.DataFrame({
        'one': pd.Series(np.random.randn(3), index=['a', 'b', 'c']),
        'two': pd.Series(np.random.randn(4), index=['a', 'b', 'c', 'd']),
        'three': pd.Series(np.random.randn(3), index=['b', 'c', 'd'])})

In [7]:
df.apply(np.mean)

one      0.011006
two      0.168440
three    0.414954
dtype: float64

In [8]:
df.apply(np.mean, axis=1)

a   -0.102897
b    0.553012
c   -0.033454
d    0.299379
dtype: float64

In [9]:
df.apply(lambda x: x.max() - x.min())

one      1.485231
two      0.431220
three    0.906500
dtype: float64

In [10]:
df.apply(np.cumsum)

,one,two,three
a,-0.603409,0.397615,NaN
b,0.278412,0.415330,0.759500
c,0.033018,0.707364,0.612499
d,NaN,0.673759,1.244862


In [11]:
df.apply(np.exp)

,one,two,three
a,0.546944,1.488271,NaN
b,2.415294,1.017873,2.137207
c,0.782396,1.339148,0.863294
d,NaN,0.966953,1.882053


The apply() method will also dispatch on a string method name.

In [14]:
df.apply('mean')

one      0.011006
two      0.168440
three    0.414954
dtype: float64

In [16]:
df.apply('mean', axis=1)

a   -0.102897
b    0.553012
c   -0.033454
d    0.299379
dtype: float64

#### The return type of the function passed to apply() affects the type of the final output from DataFrame.apply for the default behaviour:

1. If the applied function __returns a Series, the final output is a DataFrame.__ The columns match the index of the Series returned by the applied function.

2. If the applied function __returns any other type, the final output is a Series.__


This default behaviour can be __overridden using the result_type__, which accepts __three options: reduce, broadcast, and expand.__ These will determine how list-likes return values expand (or not) to a DataFrame.

apply() combined with some cleverness can be used to answer many questions about a data set. For example, suppose we wanted to extract the date where the maximum value for each column occurred:

In [18]:
tsdf = pd.DataFrame(np.random.randn(1000, 3), columns=['A', 'B', 'C'], index=pd.date_range('1/1/2000', periods=1000))

In [19]:
tsdf.apply(lambda x: x.idxmax())

A   2000-12-02
B   2001-06-19
C   2000-12-12
dtype: datetime64[ns]

You may also __pass additional arguments and keyword arguments to the apply() method__. For instance, consider the following function you would like to apply:

>def subtract_and_divide(x, sub, divide=1):
    return (x - sub) / divide

You may then apply this function as follows:

>df.apply(subtract_and_divide, args=(5,), divide=3)

Another useful feature is the __ability to pass Series methods__ to carry out some Series operation on each column or row:

In [22]:
tsdf.head()

,A,B,C
2000-01-01,-0.210459,0.653285,0.465196
2000-01-02,1.122527,0.837860,-0.377406
2000-01-03,-0.955038,0.605241,0.079956
2000-01-04,0.316868,0.860398,0.741626
2000-01-05,-0.313304,0.444202,-1.690462


In [24]:
tsdf.apply(pd.Series.interpolate).head()

,A,B,C
2000-01-01,-0.210459,0.653285,0.465196
2000-01-02,1.122527,0.837860,-0.377406
2000-01-03,-0.955038,0.605241,0.079956
2000-01-04,0.316868,0.860398,0.741626
2000-01-05,-0.313304,0.444202,-1.690462


#### Finally, apply() takes an argument 'raw' which is False by default, which converts each row or column into a Series before applying the function. When set to True, the passed function will instead receive an ndarray object, which has positive performance implications if you do not need the indexing functionality.

### Aggregation API

New in version 0.20.0.

The aggregation API allows one to __express possibly multiple aggregation operations in a single concise way__. This API is similar across pandas objects, see groupby API, the window functions API, and the resample API. The entry point for aggregation is __DataFrame.aggregate()__, or the alias __DataFrame.agg()__.

In [28]:
tsdf = pd.DataFrame(np.random.randn(10, 3), columns=['A', 'B', 'C'], index=pd.date_range('1/1/2000', periods=10))

In [29]:
tsdf.iloc[3:7] = np.nan

In [30]:
tsdf

,A,B,C
2000-01-01,1.767517,0.172360,0.692780
2000-01-02,-1.714862,-1.006860,-0.119473
2000-01-03,1.625186,0.830770,-0.248270
2000-01-04,NaN,NaN,NaN
2000-01-05,NaN,NaN,NaN
2000-01-06,NaN,NaN,NaN
2000-01-07,NaN,NaN,NaN
2000-01-08,-0.030334,-1.568667,0.053116
2000-01-09,-1.360803,-0.502043,-0.810750
2000-01-10,-0.888981,1.492465,-0.690779


Using a single function is equivalent to apply(). You can also pass __named methods as strings__. These will __return a Series__ of the aggregated output:

In [31]:
tsdf.agg(np.sum)

A   -0.602277
B   -0.581975
C   -1.123376
dtype: float64

In [33]:
tsdf.agg('sum')

A   -0.602277
B   -0.581975
C   -1.123376
dtype: float64

these are equivalent to a ``.sum()`` because we are aggregating on a single function

In [34]:
tsdf.sum()

A   -0.602277
B   -0.581975
C   -1.123376
dtype: float64

#### Single aggregations on a Series this will return a scalar value:

In [35]:
tsdf.A.agg('sum')

-0.6022774198389863

### Aggregating with multiple functions

__You can pass multiple aggregation arguments as a list.__ The results of each of the passed functions will be a row in the resulting DataFrame. These are naturally named from the aggregation function.

In [37]:
tsdf.agg(['sum'])

,A,B,C
sum,-0.602277,-0.581975,-1.123376


In [38]:
tsdf.agg(['sum', 'mean'])

,A,B,C
sum,-0.602277,-0.581975,-1.123376
mean,-0.100380,-0.096996,-0.187229


#### On a Series, multiple functions return a Series, indexed by the function names:

In [39]:
tsdf.A.agg(['sum', 'mean'])

sum    -0.602277
mean   -0.100380
Name: A, dtype: float64

Passing a lambda function will yield a __``<lambda>``__ named row:

In [40]:
tsdf.A.agg(['sum', lambda x: x.mean()])

sum        -0.602277
<lambda>   -0.100380
Name: A, dtype: float64

Passing a named function will yield that name for the row:

In [41]:
def mymean(x):
   return x.mean()

tsdf.A.agg(['sum', mymean])

sum      -0.602277
mymean   -0.100380
Name: A, dtype: float64

### Aggregating with a dict

__Passing a dictionary of column names to a scalar or a list of scalars, to DataFrame.agg__ allows you to customize which functions are applied to which columns. Note that the results are not in any particular order, you can use an __``OrderedDict`` instead to guarantee ordering.__

In [42]:
tsdf.agg({'A': 'mean', 'B': 'sum'})

A   -0.100380
B   -0.581975
dtype: float64

__Passing a list-like will generate a DataFrame output.__ You will get a matrix-like output of all of the aggregators. The output will consist of all unique functions. __Those that are not noted for a particular column will be NaN:__

In [43]:
tsdf.agg({'A': ['mean', 'min'], 'B': 'sum'})

,A,B
mean,-0.100380,NaN
min,-1.714862,NaN
sum,NaN,-0.581975


#### Mixed dtypes

When presented with mixed dtypes that __cannot aggregate, .agg will only take the valid aggregations.__ This is similar to how groupby .agg works.

In [44]:
mdf = pd.DataFrame({'A': [1, 2, 3],
                    'B': [1., 2., 3.],
                    'C': ['foo', 'bar', 'baz'],
                     'D': pd.date_range('20130101', periods=3)})

In [45]:
mdf.dtypes

A             int64
B           float64
C            object
D    datetime64[ns]
dtype: object

In [46]:
mdf.agg(['min', 'sum'])

,A,B,C,D
min,1,1.0,bar,2013-01-01
sum,6,6.0,foobarbaz,NaT


#### Custom describe

#### With .agg() is it possible to easily create a custom describe function, similar to the built in describe function.

In [48]:
from functools import partial

q_25 = partial(pd.Series.quantile, q=0.25)
q_25.__name__ = '25%'

q_75 = partial(pd.Series.quantile, q=0.75)
q_75.__name__ = '75%'

In [49]:
tsdf.agg(['sum', 'count', 'mean', 'std', q_25, 'median', q_75])

,A,B,C
sum,-0.602277,-0.581975,-1.123376
count,6.000000,6.000000,6.000000
mean,-0.100380,-0.096996,-0.187229
std,1.502573,1.150428,0.544580
25%,-1.242847,-0.880656,-0.580152
median,-0.459658,-0.164841,-0.183871
75%,1.211306,0.666167,0.009969


### Transform API

New in version 0.20.0.

#### The ``transform()`` method returns an object that is indexed the same (same size) as the original. This API allows you to provide multiple operations at the same time rather than one-by-one. Its API is quite similar to the .agg API.

In [50]:
tsdf = pd.DataFrame(np.random.randn(10, 3), columns=['A', 'B', 'C'], index=pd.date_range('1/1/2000', periods=10))

In [51]:
tsdf.iloc[3:7] = np.nan

In [52]:
tsdf

,A,B,C
2000-01-01,-0.134471,0.520961,1.171910
2000-01-02,1.360432,0.646665,0.241652
2000-01-03,0.381572,-0.076118,-1.824613
2000-01-04,NaN,NaN,NaN
2000-01-05,NaN,NaN,NaN
2000-01-06,NaN,NaN,NaN
2000-01-07,NaN,NaN,NaN
2000-01-08,0.653048,-0.931690,-1.869104
2000-01-09,1.047737,0.470698,-0.616247
2000-01-10,0.612959,0.949390,-0.159805


__Transform the entire frame.__ ``.transform()`` allows input functions as: a NumPy function, a string function name or a user defined function.

In [53]:
tsdf.transform(np.abs)

,A,B,C
2000-01-01,0.134471,0.520961,1.171910
2000-01-02,1.360432,0.646665,0.241652
2000-01-03,0.381572,0.076118,1.824613
2000-01-04,NaN,NaN,NaN
2000-01-05,NaN,NaN,NaN
2000-01-06,NaN,NaN,NaN
2000-01-07,NaN,NaN,NaN
2000-01-08,0.653048,0.931690,1.869104
2000-01-09,1.047737,0.470698,0.616247
2000-01-10,0.612959,0.949390,0.159805


In [54]:
tsdf.transform('abs')

,A,B,C
2000-01-01,0.134471,0.520961,1.171910
2000-01-02,1.360432,0.646665,0.241652
2000-01-03,0.381572,0.076118,1.824613
2000-01-04,NaN,NaN,NaN
2000-01-05,NaN,NaN,NaN
2000-01-06,NaN,NaN,NaN
2000-01-07,NaN,NaN,NaN
2000-01-08,0.653048,0.931690,1.869104
2000-01-09,1.047737,0.470698,0.616247
2000-01-10,0.612959,0.949390,0.159805


In [55]:
tsdf.transform(lambda x: x.abs())

,A,B,C
2000-01-01,0.134471,0.520961,1.171910
2000-01-02,1.360432,0.646665,0.241652
2000-01-03,0.381572,0.076118,1.824613
2000-01-04,NaN,NaN,NaN
2000-01-05,NaN,NaN,NaN
2000-01-06,NaN,NaN,NaN
2000-01-07,NaN,NaN,NaN
2000-01-08,0.653048,0.931690,1.869104
2000-01-09,1.047737,0.470698,0.616247
2000-01-10,0.612959,0.949390,0.159805


Here transform() received a single function; this is equivalent to a ufunc application.

In [56]:
np.abs(tsdf)

,A,B,C
2000-01-01,0.134471,0.520961,1.171910
2000-01-02,1.360432,0.646665,0.241652
2000-01-03,0.381572,0.076118,1.824613
2000-01-04,NaN,NaN,NaN
2000-01-05,NaN,NaN,NaN
2000-01-06,NaN,NaN,NaN
2000-01-07,NaN,NaN,NaN
2000-01-08,0.653048,0.931690,1.869104
2000-01-09,1.047737,0.470698,0.616247
2000-01-10,0.612959,0.949390,0.159805


#### Passing a single function to .transform() with a Series will yield a single Series in return.

In [57]:
tsdf.A.transform(np.abs)

2000-01-01    0.134471
2000-01-02    1.360432
2000-01-03    0.381572
2000-01-04         NaN
2000-01-05         NaN
2000-01-06         NaN
2000-01-07         NaN
2000-01-08    0.653048
2000-01-09    1.047737
2000-01-10    0.612959
Freq: D, Name: A, dtype: float64

### Transform with multiple functions

Passing multiple functions __will yield a column MultiIndexed DataFrame__. The first level will be the __original frame column names__; the second level will be the names of the __transforming functions.__

In [58]:
tsdf.transform([np.abs, lambda x: x + 1])

A                   B                   C          
            absolute  <lambda>  absolute  <lambda>  absolute  <lambda>
2000-01-01  0.134471  0.865529  0.520961  1.520961  1.171910  2.171910
2000-01-02  1.360432  2.360432  0.646665  1.646665  0.241652  1.241652
2000-01-03  0.381572  1.381572  0.076118  0.923882  1.824613 -0.824613
2000-01-04       NaN       NaN       NaN       NaN       NaN       NaN
2000-01-05       NaN       NaN       NaN       NaN       NaN       NaN
2000-01-06       NaN       NaN       NaN       NaN       NaN       NaN
2000-01-07       NaN       NaN       NaN       NaN       NaN       NaN
2000-01-08  0.653048  1.653048  0.931690  0.068310  1.869104 -0.869104
2000-01-09  1.047737  2.047737  0.470698  1.470698  0.616247  0.383753
2000-01-10  0.612959  1.612959  0.949390  1.949390  0.159805  0.840195

Passing __multiple functions to a Series will yield a DataFrame__. The resulting column names will be the transforming functions.

In [59]:
tsdf.A.transform([np.abs, lambda x: x + 1])

,absolute,<lambda>
2000-01-01,0.134471,0.865529
2000-01-02,1.360432,2.360432
2000-01-03,0.381572,1.381572
2000-01-04,NaN,NaN
2000-01-05,NaN,NaN
2000-01-06,NaN,NaN
2000-01-07,NaN,NaN
2000-01-08,0.653048,1.653048
2000-01-09,1.047737,2.047737
2000-01-10,0.612959,1.612959


### Transforming with a dict

#### Passing a dict of functions will allow selective transforming per column.

In [60]:
tsdf.transform({'A': 'abs', 'B': lambda x: x + 1})

,A,B
2000-01-01,0.134471,1.520961
2000-01-02,1.360432,1.646665
2000-01-03,0.381572,0.923882
2000-01-04,NaN,NaN
2000-01-05,NaN,NaN
2000-01-06,NaN,NaN
2000-01-07,NaN,NaN
2000-01-08,0.653048,0.068310
2000-01-09,1.047737,1.470698
2000-01-10,0.612959,1.949390


#### Passing a dict of lists will generate a MultiIndexed DataFrame with these selective transforms.

In [61]:
tsdf.transform({'A': np.abs, 'B': [lambda x: x + 1, 'sqrt']})

E:\Study\ML\Anaconda\lib\site-packages\pandas\core\base.py:315: RuntimeWarning: invalid value encountered in sqrt
  return f(self, *args, **kwargs)


A         B          
            absolute  <lambda>      sqrt
2000-01-01  0.134471  1.520961  0.721776
2000-01-02  1.360432  1.646665  0.804155
2000-01-03  0.381572  0.923882       NaN
2000-01-04       NaN       NaN       NaN
2000-01-05       NaN       NaN       NaN
2000-01-06       NaN       NaN       NaN
2000-01-07       NaN       NaN       NaN
2000-01-08  0.653048  0.068310       NaN
2000-01-09  1.047737  1.470698  0.686074
2000-01-10  0.612959  1.949390  0.974366

### Applying elementwise functions

#### Since not all functions can be vectorized (accept NumPy arrays and return another array or value), the methods applymap() on DataFrame and analogously map() on Series accept any Python function taking a single value and returning a single value. For example:

In [64]:
df4 = df.copy()
df4

,one,two,three
a,-0.603409,0.397615,NaN
b,0.881821,0.017715,0.759500
c,-0.245394,0.292033,-0.147000
d,NaN,-0.033605,0.632363


In [65]:
def f(x):
    return len(str(x))

In [66]:
df4.one.map(f)

a    19
b    18
c    20
d     3
Name: one, dtype: int64

In [67]:
df4.applymap(f)

,one,two,three
a,19,19,3
b,18,20,18
c,20,19,20
d,3,20,18


#### Series.map() has an additional feature; it can be used to easily “link” or “map” values defined by a secondary series. This is closely related to merging/joining functionality:

In [68]:
s = pd.Series(['six', 'seven', 'six', 'seven', 'six'], index=['a', 'b', 'c', 'd', 'e'])
t = pd.Series({'six': 6., 'seven': 7.})

In [71]:
s

a      six
b    seven
c      six
d    seven
e      six
dtype: object

In [73]:
t

six      6.0
seven    7.0
dtype: float64

In [74]:
t.index

Index(['six', 'seven'], dtype='object')

In [70]:
s.map(t)

a    6.0
b    7.0
c    6.0
d    7.0
e    6.0
dtype: float64